In [3]:
import os

import sys
import glob
import numpy as np
import pickle
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
from projection_utils import Projection
from data_parser import read_keypoints
import time
import torch

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device = torch.device('cpu')

sys.path.insert(1, '../')
from virtual_huams_resource.utils import get_smplx_body_model

from virtual_huams_resource.benji_prox_dataloader import *
from virtual_huams_resource.utils import *
from virtual_huams_resource.visualisation import *
from virtual_huams_resource.simple_transformer import PoseTransformer
import torch

sys.path.insert(0, "../virtual_huams_resource")
from virtual_huams_resource.benji_prox_dataloader import DatasetBase, proxDatasetSkeleton, get_start_idx
from virtual_huams_resource.pose_gru import PoseGRU_inputFC2

numpy: 1.21.5


In [2]:
import wandb
run_path = "vh-motion-pred/rnn/19ugvg0b"  # grossly overfit training
# run_path = "vh-motion-pred/rnn/3injxdum"   # best (but not great) validation score
api = wandb.Api()
run = api.run(run_path)

# save_path = 'saves/GRU_joints_15_30_3fps_24_05_1854_epoch199_bn39.pt'
save_path = 'GRU_joints_15_30_3fps_3layers512_big_29_05_1151_epoch64_bn499.pt'  # grossly overfit training
# save_path = 'saves/GRU_joints_15_30_3fps_3layers512_28_05_1945_epoch140_bn39.pt'  # best (but not great) validation score
wandb.restore(save_path, run_path=run_path)

gru = PoseGRU_inputFC2(input_size=(25,3), n_layers=3)
restore_dict = torch.load(save_path, map_location=device)
gru.load_state_dict(restore_dict['model_state_dict'])

<All keys matched successfully>

In [1]:
import wandb
# run_path = "vh-motion-pred/rnn/19ugvg0b"  # grossly overfit training
run_path = "vh-motion-pred/rnn/3injxdum"   # best (but not great) validation score
api = wandb.Api()
run = api.run(run_path)

# save_path = 'saves/GRU_joints_15_30_3fps_24_05_1854_epoch199_bn39.pt'
# save_path = 'GRU_joints_15_30_3fps_3layers512_big_29_05_1151_epoch64_bn499.pt'  # grossly overfit training
save_path = 'saves/GRU_joints_15_30_3fps_3layers512_28_05_1945_epoch140_bn39.pt'  # best (but not great) validation score
wandb.restore(save_path, run_path=run_path)

gru2 = PoseGRU_inputFC2(input_size=(25,3), n_layers=3)
restore_dict = torch.load(save_path, map_location=device)
gru2.load_state_dict(restore_dict['model_state_dict'])

NameError: name 'PoseGRU_inputFC2' is not defined

In [4]:
run_path = "vh-motion-pred/rnn/12f8x1vr"   # short sequence RNN
api = wandb.Api()
run = api.run(run_path)

save_path = 'GRU_joints_15_30_3fps_3layers512_short_30_05_2210_epoch77_bn499.pt'
wandb.restore(save_path, run_path=run_path)

gru = PoseGRU_inputFC2(input_size=(25,3), n_layers=2)
restore_dict = torch.load(save_path, map_location=device)
gru.load_state_dict(restore_dict['model_state_dict'])


<_io.TextIOWrapper name='C:\\Users\\benja\\coding\\eth\\prox\\src\\GRU_joints_15_30_3fps_3layers512_short_30_05_2210_epoch77_bn499.pt' mode='r' encoding='cp1252'>

<All keys matched successfully>

In [ ]:
run_path = "vh-motion-pred/rnn/1s3wl3lw'
api = wandb.Api()
run = api.run(run_path)

save_path = 'GRU_joints_15_30_3fps_3layers512_short_30_05_2210_epoch77_bn499.pt'
wandb.restore(save_path, run_path=run_path)

gru = PoseGRU_inputFC2(input_size=(25,3), n_layers=2)
restore_dict = torch.load(save_path, map_location=device)
gru.load_state_dict(restore_dict['model_state_dict'])


In [5]:
trans_save_path = 'transformer_best_model_30_05_1139.pt'
trans_run_path="vh-motion-pred/transformer_viz/3m5f6cyd"
model = PoseTransformer(num_tokens=25*3).to(device)
restore_dict = torch.load(trans_save_path, map_location=device)
model.load_state_dict(restore_dict['model_state_dict'])

FileNotFoundError: [Errno 2] No such file or directory: 'transformer_best_model_30_05_1139.pt'

In [ ]:

root_dir = "D:/prox_data"
# root_dir = "/cluster/scratch/bdayan/prox_data"

smplx_model_path='../models_smplx_v1_1/models/'
in_frames=15
pred_frames=30
frame_jump=10
window_overlap_factor=8
lr = 0.0001
n_iter = 100



val_areas =['BasementSittingBooth', 'N3OpenArea']

pd_train = proxDatasetSkeleton(root_dir=root_dir + '/PROXD_attempt2/PROXD', in_frames=in_frames, pred_frames=pred_frames, \
                       output_type='raw_pkls', smplx_model_path=smplx_model_path, frame_jump=frame_jump, window_overlap_factor=window_overlap_factor, extra_prefix='joints_worldnorm.pkl')

# pdc_train = DatasetBase(root_dir=root_dir + '/recordings', in_frames=in_frames, pred_frames=pred_frames,
#                                              search_prefix='Color', extra_prefix='', frame_jump=frame_jump,
#                   window_overlap_factor=window_overlap_factor)


pd_val = proxDatasetSkeleton(root_dir=root_dir + '/PROXD_attempt2/PROXD', in_frames=in_frames, pred_frames=pred_frames, \
                       output_type='raw_pkls', smplx_model_path=smplx_model_path, frame_jump=frame_jump, window_overlap_factor=window_overlap_factor, extra_prefix='joints_worldnorm.pkl')

pd_train.sequences = [seq for seq in pd_train.sequences if not any([area in seq[0] for area in val_areas])]
pd_val.sequences = [seq for seq in pd_val.sequences if any([area in seq[0] for area in val_areas])]

# pdc_val = DatasetBase(root_dir=root_dir + '/recordings', in_frames=in_frames, pred_frames=pred_frames,
#                                              search_prefix='Color', extra_prefix='', frame_jump=frame_jump,
#                   window_overlap_factor=window_overlap~_factor)

print(f'length pd_train: {len(pd_train)}; length pd_val: {len(pd_val)}')

# print(f'length pdc_train, pd_train: {len(pdc_train)}, {len(pd_train)}')
# pdc_train.align(pd_train)
# pd_train.align(pdc_train)
# print(f'length pdc_train, pd_train: {len(pdc_train)}, {len(pd_train)}')

# print(f'length pdc_val, pd_val: {len(pdc_val)}, {len(pd_val)}')
# pdc_val.align(pd_val)
# pd_val.align(pdc_val)
# print(f'length pdc, pd_val: {len(pdc_val)}, {len(pd_val)}')


# dataloader = DataLoader(pd, batch_size=batch_size,
#                         shuffle=True, num_workers=0, collate_fn=my_collate)

In [ ]:
LIMBS = [(23, 15),
         (24, 15),
         (15, 22),
         (22, 12),
         # left arm
         (12, 13),
         (13, 16),
         (16, 18),
         (18, 20),
         # right arm
         (12, 14),
         (14, 17),
         (17, 19),
         (19, 21),
         # spline
         (12, 9),
         (9, 6),
         (6, 3),
         (3, 0),
         # left leg
         (0, 1),
         (1, 4),
         (4, 7),
         (7, 10),
         # right leg
         (0, 2),
         (2, 5),
         (5, 8),
         (8, 11)]

In [ ]:
pd_val.sequences[1][0]

In [ ]:
a, b = get_start_idx(31, pd_val.bounds, pd_val.start_jump)
a,b

In [ ]:

idx = 150
pd = pd_train

_, (a, in_skels), (b, fut_skels) = pd.__getitem__(idx)

start, seq_idx = get_start_idx(idx, pd.bounds, pd.start_jump)
scene_name = pd.sequences[seq_idx][0]
scene_name = scene_name[:scene_name.index('_')]
print(scene_name)
# ______________ read point cloud ______________
scene_point_cloud = o3d.io.read_point_cloud(os.path.join(root_dir, "scenes", scene_name + ".ply"))

In [32]:
# noise = 0.07 * np.random.random(size=((len(fut_skels),) + fut_skels[0].shape))
# noise_cumsum = np.cumsum(noise, axis=0)
# pred_skels = [torch.Tensor(noise) + fut_skel for noise, fut_skel in zip(noise_cumsum, fut_skels)]
# pred_skels = [None] * len(in_skels) + pred_skels

in_skels_tensor = torch.cat(in_skels)
print(in_skels_tensor.shape)
pelvis = in_skels_tensor[0, 0, :]
print(pelvis.shape)
in_skels_world_norm = in_skels_tensor - pelvis
cur_state, pred_skels_world = gru2.forward_prediction(in_skels_world_norm.unsqueeze(0), pred_len=30)
pred_skels_world = pred_skels_world + pelvis
print(pred_skels_world.shape)
pred_skels_world = pred_skels_world.squeeze()  # (30, 25, 3)
pred_skels = [None] * len(in_skels) + [pred_skel for pred_skel in pred_skels_world]


gt_skels = in_skels + fut_skels



# gt_skels = in_skels
# pred_skels = [None] * len(in_skels)

torch.Size([15, 25, 3])
torch.Size([3])
torch.Size([1, 30, 25, 3])


In [33]:

i = -1
green = np.array([0.3, 1.0, 0.3])
red = np.array([1.0, 0.3, 0.3])

# If no pred skels set pred_skels to [None]

def forward(vis):
    # skeleton = next(skeleton_iter)          # render first frame
    global i
    i += 1
    if i == len(gt_skels):
        i = 0
    skeleton_gt = gt_skels[i]
    skeleton_pred = pred_skels[i]
    vis.clear_geometries()
    update(vis, skeleton_gt, color=green)
    if skeleton_pred is not None:
        update(vis, skeleton_pred, color=red)

    vis.poll_events()
    vis.update_renderer()
    vis.run()

def backward(vis):
    # skeleton = next(skeleton_iter)          # render first frame
    global i
    i -= 1
    if i == -1:
        i = len(gt_skels)-1
    skeleton_gt = gt_skels[i]
    skeleton_pred = pred_skels[i]
    vis.clear_geometries()
    update(vis, skeleton_gt, color=green)
    if skeleton_pred is not None:
        update(vis, skeleton_pred, color=red)

    vis.poll_events()
    vis.update_renderer()
    vis.run()

def update(vis, skeleton, color=None):
    skeleton = skeleton.detach().numpy()
    skeleton = skeleton.astype(np.float64)
    skeleton = skeleton.squeeze()
    # print(skeleton)
    skeleton_input = o3d.geometry.LineSet(
            o3d.utility.Vector3dVector(skeleton),
            o3d.utility.Vector2iVector(LIMBS))
    if color is not None:
        skeleton_input.paint_uniform_color(color)
    ctrl = vis.get_view_control()
    cam_param = ctrl.convert_to_pinhole_camera_parameters()

    vis.add_geometry(scene_point_cloud)
    vis.add_geometry(skeleton_input)
    ctrl.convert_from_pinhole_camera_parameters(cam_param)

    
# ______________ animate skeleton in point cloud ______________
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()
vis.register_key_callback(65, forward)       # press A
vis.register_key_callback(66, backward)       # press B?

# Don't press C! instead close window with mouse to avoid kernel die.
vis.register_key_callback(67, lambda vis: vis.destroy_window())
vis.add_geometry(scene_point_cloud)

# render first frame
forward(vis)

# skeleton = in_skels[0].detach().numpy()          # render first frame
# skeleton = skeleton.astype(np.float64)
# skeleton = skeleton.squeeze()
# skeleton_input = o3d.geometry.LineSet(
#             o3d.utility.Vector3dVector(skeleton),
#             o3d.utility.Vector2iVector(LIMBS))
# vis.add_geometry(skeleton_input)
# vis.run()
vis.destroy_window()